# Twisted Molybdenum Disulfide bilayers at various angles.

## 0. Introduction.

This notebook demonstrates how to generate a twisted interface between two materials using commensurate lattices. The example uses molybdenum disulfide (MoS2) as both the film and substrate materials. The notebook uses the new `create_commensurate_interface` function which first creates a slab from the material and then performs commensurate lattice matching to find valid supercells for the target twist angle. The algorithm searches for supercell matrices within specified size limits to achieve the target twist angle within tolerance. The generated interface is visualized and analyzed to determine the actual twist angle and the number of atoms in the interface.

> **Kaihui Liu, Liming Zhang, Ting Cao, Chenhao Jin, Diana Qiu, Qin Zhou, Alex Zettl, Peidong Yang, Steve G. Louie & Feng Wang**
> Evolution of interlayer coupling in twisted molybdenum disulfide bilayers. Nature Communications, 5, 4966. 2014.
 > [https://doi.org/10.1038/ncomms5966](https://doi.org/10.1038/ncomms5966)

The twisted MoS2 bilayers are shown in the following Figure 4 from the article.

<img src="https://github.com/Exabyte-io/documentation/raw/12617167278ae3523adc028583b21ea4e8ebd197/images/tutorials/materials/interfaces/twisted-bilayer-molybdenum-disulfide/MoS2-twisted-bilayers.png" alt="Twisted MoS2 bilayers" width="600"/>

## 1. Prepare the Environment
### 1.1. Set up the notebook
Let's set angles and corresponding distances for the twisted interface from the article.

In [ ]:
# Uncomment lines to reproduce specific cases from the article
INTERFACE_PARAMETERS = [
    # {"angle": 0.0, "distance": 6.8},
    # {"angle": 13.0, "distance": 6.5},
    {"angle": 22.0, "distance": 6.5},
    # {"angle": 38.0, "distance": 6.5},
    # {"angle": 47.0, "distance": 6.5},
    # {"angle": 60.0, "distance": 6.2},
]

# Slab creation parameters
MILLER_INDICES = (0, 0, 1)  # Miller indices for slab creation
NUMBER_OF_LAYERS = 1  # Number of layers in the slab

INTERFACE_VACUUM = 20.0  # in Angstroms

# Search algorithm parameters
MAX_REPETITION = None  # Maximum supercell matrix element value (None for automatic)
ANGLE_TOLERANCE = 0.5  # in degrees
RETURN_FIRST_MATCH = True  # If True, returns first solution within tolerance

# Visualization parameters
SHOW_INTERMEDIATE_STEPS = True
VISUALIZE_REPETITIONS = [3, 3, 1]

 ### 1.2. Install packages
The step executes only in Pyodide environment. For other environments, the packages should be installed via `pip install` (see [README](../../README.ipynb)).

In [ ]:
import sys

if sys.platform == "emscripten":
    import micropip

    await micropip.install("https://exabyte-io.github.io/api-examples/mat3ra_api_examples-0.1.dev1+gc41b0b8-py3-none-any.whl", deps=False)
    await micropip.install('mat3ra-utils')
    from mat3ra.utils.jupyterlite.packages import install_packages

    await install_packages("specific_examples")

### 1.3. Get input material
We'll use the MoS2 material from Standata.


In [ ]:
from mat3ra.made.material import Material
from mat3ra.standata.materials import Materials
from utils.visualize import visualize_materials

material = Material.create(Materials.get_by_name_and_categories("MoS2", "2D"))

print("Initial material properties:")
print(f"Formula: {material.formula}")
print(f"Number of atoms: {len(material.basis.elements.ids)}")

if SHOW_INTERMEDIATE_STEPS:
    visualize_materials(material, repetitions=VISUALIZE_REPETITIONS)
    visualize_materials(material, repetitions=VISUALIZE_REPETITIONS, rotation="-90x")


 ## 3. Generate Twisted Interface
 ### 3.1. Create slab


In [ ]:
from mat3ra.made.tools.modify import translate_to_z_level
from mat3ra.esse.models.core.reusable.axis_enum import AxisEnum
from mat3ra.made.tools.build.slab.helpers import create_slab
from mat3ra.made.tools.build.interface.helpers import create_commensurate_interface

slab = create_slab(
    crystal=material,
    miller_indices=MILLER_INDICES,
    use_conventional_cell=USE_CONVENTIONAL_CELL,
    use_orthogonal_c=USE_ORTHOGONAL_C,
    number_of_layers=NUMBER_OF_LAYERS,
    vacuum=0.0, # No vacuum in the slab, it is a 2D material
)
slab = translate_to_z_level(slab, "center")

visualize_materials(slab, rotation="-90x")

### 3.2. Create twisted interfaces

In [ ]:
interfaces = []
for parameters in INTERFACE_PARAMETERS:
    interface = create_commensurate_interface(
        material=slab,
        target_angle=parameters["angle"],
        angle_tolerance=ANGLE_TOLERANCE,
        max_repetition_int=MAX_REPETITION,
        return_first_match=RETURN_FIRST_MATCH,
        direction=AxisEnum.z,
        gap=parameters["distance"],
        vacuum=INTERFACE_VACUUM
    )
    interfaces.append(interface)
    print(f"Created interface with twist angle {parameters['angle']}° and {len(interface.basis.elements.ids)} atoms")


## 4. Preview the  materials


In [ ]:
from utils.visualize import visualize_materials

for interface in interfaces:
    visualize_materials(interface, viewer="wave")


## 5. Download materials


In [ ]:
from utils.jupyterlite import download_content_to_file, set_materials

set_materials(interfaces)

for idx, interface in enumerate(interfaces):
    download_content_to_file(interface, f"twisted_interface_{idx}.json")